In [1]:
import string
import re
import random
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2025-11-02 10:21:44.417884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-02 10:21:44.418003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-02 10:21:44.674546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
text_path = "/kaggle/input/englishrussian-dictionary-for-machine-translate/rus.txt"
with open(text_path) as file:
    lines = file.read().split("\n")[:-1]  

In [3]:
pairs = []
for line in lines:
    parts = line.split("\t")
    if len(parts) < 2:
        continue  # Skip malformed lines
    english, russian = parts[0], parts[1]
    russian = "[start] " + russian.strip() + " [end]"
    pairs.append((english.strip(), russian))

print(f"Total sentence pairs: {len(pairs)}")
print("Sample pair:", random.choice(pairs))


Total sentence pairs: 363386
Sample pair: ('What do you think of Tom?', '[start] Что вы думаете о Томе? [end]')


In [4]:
random.seed(42)
random.shuffle(pairs)
total = len(pairs)
num_val = num_test = total // 10
num_train = total - num_val - num_test

train_pairs = pairs[:num_train]
val_pairs = pairs[num_train:num_train + num_val]
test_pairs = pairs[num_train + num_val:]

print(f"Train: {len(train_pairs)}, Val: {len(val_pairs)}, Test: {len(test_pairs)}")

Train: 290710, Val: 36338, Test: 36338


Data dibagi secara acak (dengan seed tetap untuk reprodusibilitas) menjadi 80% pelatihan, 10% validasi, dan 10% pengujian. Sebelum vektorisasi, teks dibersihkan dari tanda baca (kecuali kurung siku) dan diubah ke huruf kecil melalui fungsi custom_standardization.

In [5]:

strip_chars = string.punctuation.replace("[", "").replace("]", "")  
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    # Escape special regex chars in punctuation
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# Vectorization layers
vocab_size = 15000  # Lebih realistis untuk dataset kecil → hindari overfit
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,  # +1 for teacher forcing
    standardize=custom_standardization,
)

# Adapt vocabulary only on training data
train_eng_texts = [pair[0] for pair in train_pairs]
train_rus_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_eng_texts)
target_vectorization.adapt(train_rus_texts)

Dua lapisan TextVectorization digunakan: satu untuk sumber (Inggris) dan satu untuk target (Rusia). Keduanya dibatasi pada vocabulari 15.000 token dan panjang urutan tetap (20 untuk sumber, 21 untuk target guna mendukung teacher forcing). Pentingnya, adaptasi vocabulari dilakukan hanya pada data pelatihan untuk mencegah kebocoran data (data leakage) ke set validasi dan uji.

In [6]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        return self.token_embeddings(inputs) + self.position_embeddings(positions)

    def get_config(self):
        return {
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        }

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]  # Expand for attention
        attn_out = self.attention(inputs, inputs, attention_mask=mask)
        x = self.layernorm_1(inputs + attn_out)
        proj_out = self.dense_proj(x)
        return self.layernorm_2(x + proj_out)

    def get_config(self):
        return {"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads}

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_causal_attention_mask(self, inputs):
        seq_len = tf.shape(inputs)[1]
        i = tf.range(seq_len)[:, None]
        j = tf.range(seq_len)
        mask = tf.cast(i >= j, dtype="int32")
        return mask[None, :, :]  # Shape: (1, T, T)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = mask[:, None, :]  # (B, 1, T)
            combined_mask = tf.minimum(padding_mask, causal_mask)
        else:
            combined_mask = causal_mask

        attn1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out1 = self.layernorm_1(inputs + attn1)

        attn2 = self.attention_2(
            query=out1, value=encoder_outputs, key=encoder_outputs, attention_mask=combined_mask
        )
        out2 = self.layernorm_2(out1 + attn2)

        proj = self.dense_proj(out2)
        return self.layernorm_3(out2 + proj)

    def get_config(self):
        return {"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads}

- PositionalEmbedding: Mengkompensasi ketiadaan informasi urutan dalam mekanisme self-attention dengan menambahkan embedding posisi ke embedding token.
- TransformerEncoder: Menerapkan multi-head self-attention diikuti jaringan feed-forward dua lapis, dengan residual connection dan LayerNormalization untuk stabilitas pelatihan.
- TransformerDecoder: Memiliki dua lapisan multi-head attention: (1) masked self-attention (menggunakan causal mask agar prediksi token ke-t hanya bergantung pada token sebelumnya), dan (2) encoder-decoder attention yang menghubungkan representasi sumber dan target.

In [7]:
# Hyperparameters
embed_dim = 256
dense_dim = 512      # Kurangi dari 2048 → lebih cepat & stabil untuk 1 epoch
num_heads = 8
batch_size = 64      # Efisien di Kaggle GPU/TPU

# Build model
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="russian")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.1)(x)  # Kurangi dropout agar tidak terlalu agresif di 1 epoch
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Dataset pipeline
def format_dataset(eng, rus):
    eng = source_vectorization(eng)
    rus = target_vectorization(rus)
    return ({"english": eng, "russian": rus[:, :-1]}, rus[:, 1:])

def make_dataset(pairs):
    eng_texts, rus_texts = zip(*pairs)
    ds = tf.data.Dataset.from_tensor_slices((list(eng_texts), list(rus_texts)))
    ds = ds.batch(batch_size)
    ds = ds.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# Callback untuk log loss & acc per batch (opsional, tapi sesuai permintaan)
class BatchLogger(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if batch % 50 == 0:  
            print(f"Batch {batch} - Loss: {logs['loss']:.4f}, Acc: {logs['accuracy']:.4f}")


history = model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=[BatchLogger()]
)

Epoch 1/15
Batch 0 - Loss: 9.6972, Acc: 0.0000
   3/4543 ━━━━━━━━━━━━━━━━━━━━ 4:18 57ms/step - accuracy: 0.2687 - loss: 8.9538

I0000 00:00:1762078941.976725      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1762078942.000642      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1762078942.032730      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1762078942.041711      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  49/4543 ━━━━━━━━━━━━━━━━━━━━ 3:05 41ms/step - accuracy: 0.6475 - loss: 5.0519Batch 50 - Loss: 3.2268, Acc: 0.7184
  99/4543 ━━━━━━━━━━━━━━━━━━━━ 3:01 41ms/step - accuracy: 0.6905 - loss: 3.9207Batch 100 - Loss: 2.5085, Acc: 0.7435
 149/4543 ━━━━━━━━━━━━━━━━━━━━ 2:59 41ms/step - accuracy: 0.7105 - loss: 3.3959Batch 150 - Loss: 2.2193, Acc: 0.7563
 199/4543 ━━━━━━━━━━━━━━━━━━━━ 2:56 41ms/step - accuracy: 0.7230 - loss: 3.0804Batch 200 - Loss: 2.0598, Acc: 0.7645
 249/4543 ━━━━━━━━━━━━━━━━━━━━ 2:54 41ms/step - accuracy: 0.7320 - loss: 2.8642Batch 250 - Loss: 1.9452, Acc: 0.7713
 299/4543 ━━━━━━━━━━━━━━━━━━━━ 2:52 41ms/step - accuracy: 0.7390 - loss: 2.7037Batch 300 - Loss: 1.8646, Acc: 0.7762
 349/4543 ━━━━━━━━━━━━━━━━━━━━ 2:50 41ms/step - accuracy: 0.7446 - loss: 2.5791Batch 350 - Loss: 1.8020, Acc: 0.7803
 399/4543 ━━━━━━━━━━━━━━━━━━━━ 2:48 41ms/step - accuracy: 0.7493 - loss: 2.4784Batch 400 - Loss: 1.7478, Acc: 0.7840
 449/4543 ━━━━━━━━━━━━━━━━━━━━ 2:46 41ms/step - accuracy: 0.7533 

W0000 00:00:1762079119.197021      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1762079119.214001      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4149/4543 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.8136 - loss: 1.3777Batch 4150 - Loss: 1.0596, Acc: 0.8369
4199/4543 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.8139 - loss: 1.3739Batch 4200 - Loss: 1.0568, Acc: 0.8372
4249/4543 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step - accuracy: 0.8142 - loss: 1.3701Batch 4250 - Loss: 1.0539, Acc: 0.8374
4299/4543 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.8145 - loss: 1.3665Batch 4300 - Loss: 1.0508, Acc: 0.8377
4349/4543 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.8147 - loss: 1.3628Batch 4350 - Loss: 1.0482, Acc: 0.8379
4399/4543 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.8150 - loss: 1.3592Batch 4400 - Loss: 1.0456, Acc: 0.8381
4449/4543 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.8152 - loss: 1.3557Batch 4450 - Loss: 1.0428, Acc: 0.8384
4499/4543 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.8155 - loss: 1.3522Batch 4500 - Loss: 1.0399, Acc: 0.8386
4543/4543 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8157 - los

W0000 00:00:1762079138.338859      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1762079143.202420      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4543/4543 ━━━━━━━━━━━━━━━━━━━━ 220s 45ms/step - accuracy: 0.8157 - loss: 1.3491 - val_accuracy: 0.8645 - val_loss: 0.7594
Epoch 2/15
Batch 0 - Loss: 0.9640, Acc: 0.8367
  49/4543 ━━━━━━━━━━━━━━━━━━━━ 3:01 40ms/step - accuracy: 0.8607 - loss: 0.7906Batch 50 - Loss: 0.7817, Acc: 0.8607
  99/4543 ━━━━━━━━━━━━━━━━━━━━ 2:59 40ms/step - accuracy: 0.8607 - loss: 0.7858Batch 100 - Loss: 0.7828, Acc: 0.8607
 149/4543 ━━━━━━━━━━━━━━━━━━━━ 2:59 41ms/step - accuracy: 0.8609 - loss: 0.7840Batch 150 - Loss: 0.7789, Acc: 0.8611
 199/4543 ━━━━━━━━━━━━━━━━━━━━ 2:57 41ms/step - accuracy: 0.8610 - loss: 0.7826Batch 200 - Loss: 0.7795, Acc: 0.8612
 249/4543 ━━━━━━━━━━━━━━━━━━━━ 2:55 41ms/step - accuracy: 0.8611 - loss: 0.7813Batch 250 - Loss: 0.7728, Acc: 0.8621
 299/4543 ━━━━━━━━━━━━━━━━━━━━ 2:53 41ms/step - accuracy: 0.8613 - loss: 0.7797Batch 300 - Loss: 0.7714, Acc: 0.8623
 349/4543 ━━━━━━━━━━━━━━━━━━━━ 2:51 41ms/step - accuracy: 0.8614 - loss: 0.7786Batch 350 - Loss: 0.7728, Acc: 0.8621
 399/4543 ━━━

Model memakai optimizer RMSprop (learning rate = 0.001) dan fungsi kerugian sparse categorical crossentropy, fungsi kerugian sparse categorical crossentropy, sesuai untuk prediksi indeks token diskrit. Pipeline data dibangun menggunakan tf.data untuk efisiensi: batching, transformasi input/output melalui format_dataset (yang menerapkan teacher forcing dengan menggeser target satu langkah), serta prefetching untuk paralelisasi.

Model dilatih dengan callback BatchLogger yang mencatat training loss dan accuracy setiap 50 batch. Hasil akhir training 91.58%

In [8]:
# Prepare lookup for decoding
target_vocab = target_vectorization.get_vocabulary()
index_to_word = dict(enumerate(target_vocab))

def decode_sequence(input_sentence, max_len=20):
    # Tokenize input
    tokenized_input = source_vectorization([input_sentence])
    decoded = "[start]"

    for i in range(max_len):
        # Tokenize current output (teacher forcing tidak dipakai saat inferensi)
        target_seq = target_vectorization([decoded])[:, :-1]
        # Predict next token
        preds = model([tokenized_input, target_seq])
        next_token_idx = tf.argmax(preds[0, i, :]).numpy()
        next_word = index_to_word.get(next_token_idx, "[UNK]")
        decoded += " " + next_word
        if next_word == "[end]" or next_word == "":
            break
    # Clean output
    return decoded.replace("[start]", "").replace("[end]", "").strip()

# Test translation
test_sentences = [
    "tom knows what we need",
    "how is it going",
    "how are you",
    "it was nice seeing you",
    "till next time",
    "go"
]

print("\n=== TRANSLATION RESULTS ===")
for eng in test_sentences:
    rus_pred = decode_sequence(eng)
    print(f"EN: {eng}")
    print(f"RU: {rus_pred}\n")


=== TRANSLATION RESULTS ===
EN: tom knows what we need
RU: Том знает что нам нужно

EN: how is it going
RU: Как

EN: how are you
RU: Как ты

EN: it was nice seeing you
RU: [UNK] было видеть тебя

EN: till next time
RU: [UNK] следующий раз

EN: go
RU: [UNK]

